In [16]:
import pandas as pd 
import yfinance as yf
import math
import numpy as np 
import matplotlib as mlp
import matplotlib.pyplot as plt 
import seaborn as sns


#List of stocks/etfs to be used 
tickers_list = ['CRM', 'NVDA', 'ADBE', 'GS', 'PG', 'SMCI', 'NEE']
tickers_list.sort()
etf = ['SPY','IWM','DIA']

#Different type of data pulls for Table 1
data = yf.download(tickers_list, period='10y')#General All data
ACdata = yf.download(tickers_list, period = '3mo')['Adj Close'] #Adjusted Close 3 month trail
etf_ticker_data = yf.download(tickers_list + etf, period='1y')['Adj Close'] #Adjusted Close 12 month trail including ETFs
Datahigh = yf.download(tickers_list, period='1y')['High']
Datalow = yf.download(tickers_list, period='1y')['Low']

[*********************100%%**********************]  7 of 7 completed
[*********************100%%**********************]  7 of 7 completed
[*********************100%%**********************]  10 of 10 completed
[*********************100%%**********************]  7 of 7 completed
[*********************100%%**********************]  7 of 7 completed


In [17]:
##Portfolio Weight
Adj_Closing_Prices= data['Adj Close'].iloc[-1]
total_ACP = Adj_Closing_Prices.sum()

portfolio_weight = []

for Prices in Adj_Closing_Prices:
    portfolio_weight_raw = (Prices/total_ACP)*100
    portfolio_weight.append(round(portfolio_weight_raw,2))

#Portfolio Weight Equally Distributed? 
n = len(tickers_list)
weight = 1/n


In [18]:

#Annualized Volatility 

returns = ACdata.pct_change().dropna() 
volatility_raw = returns.std()*(np.sqrt(252))*100 
annualized_volatility = round(volatility_raw,2)


In [19]:

#Function to use for each ETF to compare to Beta(Tickers/stocks)
def against_etf(etf_symbol):
    beta_values = []

    for ticker in tickers_list:
        # Extract daily returns for the current stock and ETF
        stock_returns = etf_ticker_data[ticker].pct_change().dropna()
        etf_returns = etf_ticker_data[etf_symbol].pct_change().dropna()

        # Calculate covariance and variance
        cov_matrix = np.cov(stock_returns, etf_returns, ddof=0)
        stock_etf_cov = cov_matrix[0, 1]
        etf_variance = np.var(etf_returns, ddof=0)

        # Calculate beta
        beta = round((stock_etf_cov / etf_variance),2)

        beta_values.append(beta)

    return beta_values
    

# Calculate beta for each ETF
Beta_SPY = against_etf('SPY')
Beta_IWM = against_etf('IWM')
Beta_DIA = against_etf('DIA')

In [20]:
#Average Weekly Drop Down 
avg_weekly_dd = []

for ticker in tickers_list:
    highsum = Datahigh[ticker].sum()
    lowsum = Datalow[ticker].sum()
    
    high = highsum/52
    low = lowsum/52

    awdcalc = ((low - high)/high)*100
    avg_weekly_dd.append(round(awdcalc , 2))


In [21]:
#Same code differnt data function (note .max and .min)/ Maximum Weekly Drawdown 
max_weekly_dd = []
for ticker in tickers_list:
    highsummax = Datahigh[ticker].max()
    lowsummin = Datalow[ticker].min()
    
    highmax = highsummax/52
    lowmin = lowsummin/52

    awdcalchm = ((lowmin - highmax)/highmax)*100 
    max_weekly_dd.append(round(awdcalchm, 2))


In [22]:
#Total Return 
Total_Return = []
for ticker in tickers_list:
    datatr = yf.download(ticker, period='10y')  
    beginning_value = datatr['Adj Close'].iloc[0]
    ending_value = datatr['Adj Close'].iloc[-1]

    return_value = ((ending_value - beginning_value)/beginning_value)*100
    Total_Return.append(round(return_value,2))

#Annualized Return dependent on Total Return 
Annualized_Return = []
for i in Total_Return:
   AR = (i/10)*100
   Annualized_Return.append(round(AR,2))


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [23]:
#Creating a table 
Table1 = {'Ticker' : tickers_list ,
          'Portfolio Weight' : portfolio_weight, 
          'Portfolio Weight Equally Weighted':weight,
          'Annualized Volatility': annualized_volatility,
          'Beta against SPY' : Beta_SPY,
          'Beta against IWM': Beta_IWM ,
          'Beta against DIA': Beta_DIA, 
          'Average Weekly Drawdown':avg_weekly_dd,
          'Maximum Weekly Drawdown': max_weekly_dd,
          'Total Return': Total_Return,
          'Annualized Total Return': Annualized_Return}

df1 = pd.DataFrame(Table1)

df1

,Ticker,Portfolio Weight,Portfolio Weight Equally Weighted,Annualized Volatility,Beta against SPY,Beta against IWM,Beta against DIA,Average Weekly Drawdown,Maximum Weekly Drawdown,Total Return,Annualized Total Return
ADBE,ADBE,26.18,0.142857,27.42,1.52,0.93,1.52,-2.58,-51.49,901.61,9016.1
CRM,CRM,10.31,0.142857,22.78,1.33,0.90,1.39,-2.46,-46.96,282.39,2823.9
GS,GS,16.18,0.142857,20.21,0.93,0.78,1.24,-1.99,-26.14,141.08,1410.8
NEE,NEE,2.67,0.142857,29.73,0.79,0.54,0.96,-2.07,-41.53,237.94,2379.4
NVDA,NVDA,22.68,0.142857,39.28,2.15,1.30,1.87,-3.57,-78.49,12163.89,121638.9
PG,PG,7.34,0.142857,13.00,0.49,0.30,0.65,-1.36,-22.86,154.46,1544.6
SMCI,SMCI,14.64,0.142857,73.70,1.36,0.97,1.25,-5.72,-85.92,1980.75,19807.5


In [24]:
# Table 2 
# Data Pulls
Data10y = yf.download(tickers_list + etf, period='10y')['Adj Close']
returns_df = Data10y.pct_change().dropna()
portfolio_returns = returns_df[tickers_list].sum(axis=1)

#Data pull for Sharpe Ratio
Data1y = yf.download(tickers_list + etf, period='1y')['Adj Close']
returns_df1y = Data1y.pct_change().dropna()
portfolio_returns1y = returns_df1y[tickers_list].sum(axis=1)
etf_returns1y = returns_df1y[etf].sum(axis=1)


[*********************100%%**********************]  10 of 10 completed


[0.9, 0.82, 0.84]

In [25]:
# Correlation Against ETF
correlation_etf = []

for i in etf:
    correlation = portfolio_returns.corr(returns_df[i])
    correlation_etf.append(round(correlation, 2))


[*********************100%%**********************]  10 of 10 completed


In [ ]:
##(https://www.geeksforgeeks.org/python-numpy-cov-function/)
# Covariance Against ETF
Covariance_etf =[]

for i in etf:
    covariance = portfolio_returns.cov(returns_df[i])
    Covariance_etf.append(round(covariance, 6))


In [ ]:
#tracking error
tracking_error = []
for i in etf:
    trackingerr = portfolio_returns - returns_df[i] 
    trackingerrstd = trackingerr.std()*100
    tracking_error.append(round(trackingerrstd,2))


In [ ]:

sharpe_ratio = []
# Portfolio return, risk-free rate, and excess return
risk_free_rate = 0.0459

for i in etf:
    excess_return = portfolio_returns1y - risk_free_rate
    std_dev_excess_return = excess_return.std()
    sharpe_ratio_calc = excess_return.mean() / std_dev_excess_return
    sharpe_ratio.append(sharpe_ratio_calc)


In [ ]:
Annualized_Volatility_etf = []

for i in etf:
    etf_returns = returns_df[i]
    etf_volatility_raw = etf_returns.std() * np.sqrt(252) * 100
    etf_annualized_volatility = round(etf_volatility_raw, 2)
    Annualized_Volatility_etf.append(etf_annualized_volatility)

# Calculate the annualized volatility spread
Annualized_Volatility_spread = []

for i in range(len(etf)):
    annualized_volatility_b = annualized_volatility.std()
    spread = annualized_volatility_b - Annualized_Volatility_etf[i]
    Annualized_Volatility_spread.append(round(spread, 2))


In [ ]:
#Correlation Matrix

#Correlation Matrix(https://www.geeksforgeeks.org/create-a-correlation-matrix-using-python/)

# Calculate the correlation matrix
correlation_matrix = returns_df.corr()

# Select relevant correlations for the equal-weighted portfolio, 3 ETFs, and 7 stocks
selected_correlations = correlation_matrix.loc[tickers_list + etf, tickers_list + etf]
plt.title('Correlation Heatmap')

dataplot = sns.heatmap(selected_correlations, annot=True, cmap='vlag')
plt.show()